## Logistic regression with Scikit-learn (HR-analysis) 006

ในบทนี้ เราจะทำการวิเคราะห์ปัจจัยอะไรที่ส่งผลให้ทำคนลาออกจากงาน โดยผลลัพธ์ของเรามี 2 อย่างคือ yes = ลาออก, No = อยู่ ซึ่งข้อมูลมากมาย ในที่นี้ เราจำเป็นที่จะต้องวิเคราะห์ว่า ปัจจัยอะไรบ้างที่ทำให้คนลาออก เพื่อที่เราจะได้ปรับปรุงปัจจัยนั้นๆ ป้องกันไม่ให้คนลาออก

และในที่นี้ เราต้องการที่จะหา model ที่สามารถนำไปใช้กับข้อมูลใหม่ๆด้วย ดังนั้นเราจะมีการ split ข้อมูลออกเป็น training set และ testing set ด้วย

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv('C:\\Users\\LENOVO\\Desktop\\files for example\\HR analytics\\general_data.csv',  parse_dates=True, index_col='EmployeeID')
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeID,,,,,,,,,,,,,,,,,,,,,
1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4
3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,...,1.0,Y,15,8,3,5.0,2,5,0,3
4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,...,3.0,Y,11,8,3,13.0,5,8,7,5
5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4410
Data columns (total 23 columns):
Age                        4410 non-null int64
Attrition                  4410 non-null object
BusinessTravel             4410 non-null object
Department                 4410 non-null object
DistanceFromHome           4410 non-null int64
Education                  4410 non-null int64
EducationField             4410 non-null object
EmployeeCount              4410 non-null int64
Gender                     4410 non-null object
JobLevel                   4410 non-null int64
JobRole                    4410 non-null object
MaritalStatus              4410 non-null object
MonthlyIncome              4410 non-null int64
NumCompaniesWorked         4391 non-null float64
Over18                     4410 non-null object
PercentSalaryHike          4410 non-null int64
StandardHours              4410 non-null int64
StockOptionLevel           4410 non-null int64
TotalWorkingYears          4401 non-n

In [4]:
stocks = ['general_data', 'employee_survey_data']
dfs = []
location_prefix = 'C:\\Users\\LENOVO\\Desktop\\files for example\\HR analytics\\'
dfs = [pd.read_csv(f'{location_prefix}{s}.csv', parse_dates=True, index_col='EmployeeID') for s in stocks]

In [5]:
type(dfs)

list

In [6]:
df=pd.concat(dfs,axis=1)
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,,,,,,,,,,,,,,,,,,,
1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,...,8,0,1.0,6,1,0,0,3.0,4.0,2.0
2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,...,8,1,6.0,3,5,1,4,3.0,2.0,4.0
3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,...,8,3,5.0,2,5,0,3,2.0,2.0,1.0
4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,...,8,3,13.0,5,8,7,5,4.0,4.0,3.0
5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,...,8,2,9.0,2,6,0,4,4.0,1.0,3.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4410
Data columns (total 26 columns):
Age                        4410 non-null int64
Attrition                  4410 non-null object
BusinessTravel             4410 non-null object
Department                 4410 non-null object
DistanceFromHome           4410 non-null int64
Education                  4410 non-null int64
EducationField             4410 non-null object
EmployeeCount              4410 non-null int64
Gender                     4410 non-null object
JobLevel                   4410 non-null int64
JobRole                    4410 non-null object
MaritalStatus              4410 non-null object
MonthlyIncome              4410 non-null int64
NumCompaniesWorked         4391 non-null float64
Over18                     4410 non-null object
PercentSalaryHike          4410 non-null int64
StandardHours              4410 non-null int64
StockOptionLevel           4410 non-null int64
TotalWorkingYears          4401 non-n

In [8]:
df.Attrition.value_counts()

No     3699
Yes     711
Name: Attrition, dtype: int64

In [9]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'Gender', 'JobLevel',
       'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'Over18', 'PercentSalaryHike', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'],
      dtype='object')

In [10]:
df2=df.drop(['BusinessTravel', 'Department' , 'EducationField', 'Gender', 'Over18' ,'JobRole', 'MaritalStatus','NumCompaniesWorked','TotalWorkingYears','EmployeeCount','Education','StandardHours'], axis=1)
df2.head()

,Age,Attrition,DistanceFromHome,JobLevel,MonthlyIncome,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,,,,,,,,,,,,
1,51,No,6,1,131160,11,0,6,1,0,0,3.0,4.0,2.0
2,31,Yes,10,1,41890,23,1,3,5,1,4,3.0,2.0,4.0
3,32,No,17,4,193280,15,3,2,5,0,3,2.0,2.0,1.0
4,38,No,2,3,83210,11,3,5,8,7,5,4.0,4.0,3.0
5,32,No,10,1,23420,12,2,2,6,0,4,4.0,1.0,3.0


In [11]:
df2.columns

Index(['Age', 'Attrition', 'DistanceFromHome', 'JobLevel', 'MonthlyIncome',
       'PercentSalaryHike', 'StockOptionLevel', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'],
      dtype='object')

In [1]:
def label2val(s):
    d = {'No': 0, 'Yes': 1 }
    if s in d.keys():
        v = d[s]
    else:
        v = np.nan
    return v

In [2]:
label2val('Yes')

1

In [14]:
df2['Attrition'].apply(label2val)

EmployeeID
1       0
2       1
3       0
4       0
5       0
6       0
7       1
8       0
9       0
10      0
11      0
12      0
13      0
14      1
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      1
30      0
       ..
4381    0
4382    1
4383    0
4384    0
4385    0
4386    0
4387    1
4388    0
4389    1
4390    0
4391    0
4392    1
4393    0
4394    0
4395    0
4396    0
4397    0
4398    0
4399    0
4400    0
4401    0
4402    0
4403    1
4404    0
4405    0
4406    0
4407    0
4408    0
4409    0
4410    0
Name: Attrition, Length: 4410, dtype: int64

In [15]:
df2[['Attrition']].applymap(label2val)

,Attrition
EmployeeID,
1,0
2,1
3,0
4,0
5,0
6,0
7,1
8,0
9,0


In [16]:
likert_cols = ['Attrition']
df2[likert_cols]=df2[likert_cols].applymap(label2val)

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4410
Data columns (total 14 columns):
Age                        4410 non-null int64
Attrition                  4410 non-null int64
DistanceFromHome           4410 non-null int64
JobLevel                   4410 non-null int64
MonthlyIncome              4410 non-null int64
PercentSalaryHike          4410 non-null int64
StockOptionLevel           4410 non-null int64
TrainingTimesLastYear      4410 non-null int64
YearsAtCompany             4410 non-null int64
YearsSinceLastPromotion    4410 non-null int64
YearsWithCurrManager       4410 non-null int64
EnvironmentSatisfaction    4385 non-null float64
JobSatisfaction            4390 non-null float64
WorkLifeBalance            4372 non-null float64
dtypes: float64(3), int64(11)
memory usage: 516.8 KB


In [18]:
df2.dropna(inplace=True)
df2

,Age,Attrition,DistanceFromHome,JobLevel,MonthlyIncome,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,,,,,,,,,,,,
1,51,0,6,1,131160,11,0,6,1,0,0,3.0,4.0,2.0
2,31,1,10,1,41890,23,1,3,5,1,4,3.0,2.0,4.0
3,32,0,17,4,193280,15,3,2,5,0,3,2.0,2.0,1.0
4,38,0,2,3,83210,11,3,5,8,7,5,4.0,4.0,3.0
5,32,0,10,1,23420,12,2,2,6,0,4,4.0,1.0,3.0
6,46,0,8,4,40710,13,0,5,7,7,7,3.0,2.0,2.0
7,28,1,11,2,58130,20,1,2,0,0,0,1.0,3.0,1.0
8,29,0,18,2,31430,22,3,2,0,0,0,1.0,2.0,3.0
9,31,0,1,3,20440,21,0,2,9,7,8,2.0,4.0,3.0


In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4327 entries, 1 to 4409
Data columns (total 14 columns):
Age                        4327 non-null int64
Attrition                  4327 non-null int64
DistanceFromHome           4327 non-null int64
JobLevel                   4327 non-null int64
MonthlyIncome              4327 non-null int64
PercentSalaryHike          4327 non-null int64
StockOptionLevel           4327 non-null int64
TrainingTimesLastYear      4327 non-null int64
YearsAtCompany             4327 non-null int64
YearsSinceLastPromotion    4327 non-null int64
YearsWithCurrManager       4327 non-null int64
EnvironmentSatisfaction    4327 non-null float64
JobSatisfaction            4327 non-null float64
WorkLifeBalance            4327 non-null float64
dtypes: float64(3), int64(11)
memory usage: 507.1 KB


In [20]:
df2['EnvironmentSatisfaction'] =df2['EnvironmentSatisfaction'] .astype(int)
df2['JobSatisfaction'] =df2['JobSatisfaction'] .astype(int)
df2['WorkLifeBalance'] =df2['WorkLifeBalance'] .astype(int)

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4327 entries, 1 to 4409
Data columns (total 14 columns):
Age                        4327 non-null int64
Attrition                  4327 non-null int64
DistanceFromHome           4327 non-null int64
JobLevel                   4327 non-null int64
MonthlyIncome              4327 non-null int64
PercentSalaryHike          4327 non-null int64
StockOptionLevel           4327 non-null int64
TrainingTimesLastYear      4327 non-null int64
YearsAtCompany             4327 non-null int64
YearsSinceLastPromotion    4327 non-null int64
YearsWithCurrManager       4327 non-null int64
EnvironmentSatisfaction    4327 non-null int32
JobSatisfaction            4327 non-null int32
WorkLifeBalance            4327 non-null int32
dtypes: int32(3), int64(11)
memory usage: 456.4 KB


In [22]:
df2.head()

,Age,Attrition,DistanceFromHome,JobLevel,MonthlyIncome,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,,,,,,,,,,,,
1,51,0,6,1,131160,11,0,6,1,0,0,3,4,2
2,31,1,10,1,41890,23,1,3,5,1,4,3,2,4
3,32,0,17,4,193280,15,3,2,5,0,3,2,2,1
4,38,0,2,3,83210,11,3,5,8,7,5,4,4,3
5,32,0,10,1,23420,12,2,2,6,0,4,4,1,3


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [24]:
test_size=0.3
X_train, X_test, y_train, y_test = train_test_split (df2[['Age', 'DistanceFromHome', 'JobLevel', 'MonthlyIncome', 'PercentSalaryHike', 'StockOptionLevel',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'YearsWithCurrManager','EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']], 
    df2.Attrition,  test_size=test_size, random_state=7)

In [25]:
X_train.head()

,Age,DistanceFromHome,JobLevel,MonthlyIncome,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
EmployeeID,,,,,,,,,,,,,
503,53,8,2,120610,20,1,2,15,6,12,3,2,3
3432,52,5,4,14200,13,1,2,32,6,9,1,4,3
3329,53,9,1,20890,16,0,2,2,2,2,1,4,3
1307,32,15,1,101690,14,0,6,11,5,7,2,3,3
2510,26,2,3,65780,15,2,3,5,1,4,4,2,3


In [26]:
y_train.head()

EmployeeID
503     0
3432    1
3329    0
1307    0
2510    1
Name: Attrition, dtype: int64

In [27]:
model = LogisticRegression()
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
model.fit(X_train, y_train)

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
model.predict_proba(X_test)

array([[0.78913645, 0.21086355],
       [0.82579703, 0.17420297],
       [0.86864043, 0.13135957],
       ...,
       [0.75939046, 0.24060954],
       [0.8560417 , 0.1439583 ],
       [0.84688892, 0.15311108]])

In [30]:
model.predict_proba(X_test)[:,1]

array([0.21086355, 0.17420297, 0.13135957, ..., 0.24060954, 0.1439583 ,
       0.15311108])

In [31]:
model.score(X_train, y_train)

0.8391677675033025

In [32]:
predicted= model.predict(X_test)
predicted

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [33]:
from sklearn import metrics

In [34]:
metrics.confusion_matrix(y_test,predicted)

array([[1085,    0],
       [ 214,    0]], dtype=int64)

In [35]:
y_test.values

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [36]:
predicted

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [37]:
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      1085
           1       0.00      0.00      0.00       214

   micro avg       0.84      0.84      0.84      1299
   macro avg       0.42      0.50      0.46      1299
weighted avg       0.70      0.84      0.76      1299



C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
metrics.accuracy_score(y_test, predicted)

0.8352578906851424

In [45]:
TN, FP, FN, TP = metrics.confusion_matrix(y_test, predicted).ravel()

In [46]:
TN

1085

In [47]:
FP

0

In [48]:
FN

214

In [49]:
TP

0

In [50]:
Acurracy=(TP+TN)/(TP+TN+FP+FN)
Acurracy

0.8352578906851424